In [29]:
import pandas as pd
import numpy as np

folder = '../trictrac_database'

In [64]:
# avis_clean = pd.read_csv(f'{folder}/avis_clean.csv', header=None, names=["Game id", "User id", "Game name UI", "Username", "Datetime", "Rating", "Comment title", "Comment body"]).drop_duplicates()
# users = pd.read_csv(f'{folder}/users.csv', header=None, names = ["Username", "User id"]).drop_duplicates()
# avis_clean["Datetime"] = pd.to_datetime(avis_clean["Datetime"]) # convert from object to datetime
jeux_clean = pd.read_csv(f'{folder}/jeux_clean.csv', header=None, names = ["Game id", "Game name website", "Game name UI", "Game name year", "Description", "Type", "Extra"])
jeux_clean[jeux_clean["Game id"] == 99]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Extra
99,99,6-qui-prend-anniversaire,6 Qui prend ! anniversaire,6 Qui prend ! anniversaire (2014),Ces drôles de cartes valent de 1 à 7 «têtes de...,Ambiance|Bluff,2 à 10|10 ans et +|30


In [65]:

# Separate extra info (number of player, age categories, game duration), replace ~ by NaN
jeux_clean[["Number of players", "Age", "Game duration"]] = jeux_clean.pop("Extra").str.split('|',n=3, expand=True, regex=False).replace('~', np.nan)
jeux_clean[jeux_clean["Game id"] == 99]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Number of players,Age,Game duration
99,99,6-qui-prend-anniversaire,6 Qui prend ! anniversaire,6 Qui prend ! anniversaire (2014),Ces drôles de cartes valent de 1 à 7 «têtes de...,Ambiance|Bluff,2 à 10,10 ans et +,30


In [66]:
# Number of players separation
jeux_clean["Number of players"] = jeux_clean["Number of players"].replace(r"jusqu\'à (\d+)", r"1 à \1", regex=True) # 'jusqu'à num' to '1 à num'
jeux_clean["Number of players"] = jeux_clean["Number of players"].replace(r"à partir de (\d+)", r"\1 à 0", regex=True) # 'à partir de num' to 'num à 0'
jeux_clean["Number of players"] = jeux_clean["Number of players"].replace(r"^\s*(\d+)\s*$", r"\1 à \1", regex=True) # 'num' to 'num à num'

jeux_clean[["Min number of players", "Max number of players"]] = jeux_clean.pop("Number of players").str.extract(r"(\d+)\s*à\s*(\d+)")
# Set types
jeux_clean["Min number of players"] = jeux_clean["Min number of players"].astype('Int16')
jeux_clean["Max number of players"] = jeux_clean["Max number of players"].astype('Int16')
jeux_clean["Max number of players"] = jeux_clean["Max number of players"].replace(0, np.nan,  regex=False)
jeux_clean[jeux_clean["Game id"] == 99]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Age,Game duration,Min number of players,Max number of players
99,99,6-qui-prend-anniversaire,6 Qui prend ! anniversaire,6 Qui prend ! anniversaire (2014),Ces drôles de cartes valent de 1 à 7 «têtes de...,Ambiance|Bluff,10 ans et +,30,2,10


In [67]:
jeux_clean

,Game id,Game name website,Game name UI,Game name year,Description,Type,Age,Game duration,Min number of players,Max number of players
0,0,fever,$ Fever,$ Fever (1990),Le jeu comprend 6 dés dont les faces représent...,Jeux de dés,8 ans et +,20,2,6
1,1,cetera,&cetera,&cetera (2003),&cetera est une extension de Roads & Boats qui...,Jeux de plateau,14 à 120,NaN,1,6
2,2,43,'43,'43 (1981),'43 simule le débarquement en Sicile des force...,Jeux de guerre,12 ans et +,240,2,4
3,3,und-tschuss,...Und Tschüss,...Und Tschüss (1997),But du jeu Remporter les cartes les plus forte...,Jeux de cartes,10 ans et +,30,4,6
4,4,011,011,011 (2011),"L’histoire... Nous sommes en l’an 011, au XIX ...",Jeux de plateau,12 ans et +,150,3,6
...,...,...,...,...,...,...,...,...,...,...
10544,10544,ilots,Îlots,Îlots (2010),Tout d'abord vous créez votre plateau de jeu g...,Jeux de plateau,12 ans et +,20,2,4
10545,10545,euuromind,€uromind,€uromind (1998),Le but du jeu est de faire le tour de l'Europe...,Jeux de plateau,8 ans et +,60,2,6
10546,10546,gokiburipoka,ごきぶりポーカー,ごきぶりポーカー (2011),Un petit jeu de bluff où il faut éviter de réc...,"Animaux|Bluff|Collection / famille|Hasard (Dé,...",8 ans et +,25,2,6
10547,10547,siehui-shephy,シェフィ - Shephy,シェフィ - Shephy (2013),Contenu : 72 cartes 1 mouton noir22 Actions7 p...,Animaux|Combinaison|Pioche|Programmation,NaN,15,1,1


In [68]:
# Age sepration
jeux_clean["Age"] = jeux_clean["Age"].replace(r"(\d+) ans et \+", r"\1 à 99", regex=True) # max age is fixed to 99
jeux_clean[["Age min", "Age max"]] = jeux_clean.pop("Age").str.extract(r"(\d+) à (\d+)")

In [69]:
# Type conversion
jeux_clean["Age max"] = jeux_clean["Age max"].astype('Int64')
jeux_clean["Age min"] = jeux_clean["Age min"].astype('Int64')
jeux_clean["Game duration"] = jeux_clean["Game duration"].astype('Int64')
jeux_clean.loc[jeux_clean["Age max"] > 99, "Age max"] = 99
jeux_clean[jeux_clean["Game id"] == 99]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
99,99,6-qui-prend-anniversaire,6 Qui prend ! anniversaire,6 Qui prend ! anniversaire (2014),Ces drôles de cartes valent de 1 à 7 «têtes de...,Ambiance|Bluff,30,2,10,10,99


In [70]:
print(jeux_clean.shape)
jeux_clean.drop_duplicates()


(10549, 11)


,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
0,0,fever,$ Fever,$ Fever (1990),Le jeu comprend 6 dés dont les faces représent...,Jeux de dés,20,2,6,8,99
1,1,cetera,&cetera,&cetera (2003),&cetera est une extension de Roads & Boats qui...,Jeux de plateau,<NA>,1,6,14,99
2,2,43,'43,'43 (1981),'43 simule le débarquement en Sicile des force...,Jeux de guerre,240,2,4,12,99
3,3,und-tschuss,...Und Tschüss,...Und Tschüss (1997),But du jeu Remporter les cartes les plus forte...,Jeux de cartes,30,4,6,10,99
4,4,011,011,011 (2011),"L’histoire... Nous sommes en l’an 011, au XIX ...",Jeux de plateau,150,3,6,12,99
...,...,...,...,...,...,...,...,...,...,...,...
10544,10544,ilots,Îlots,Îlots (2010),Tout d'abord vous créez votre plateau de jeu g...,Jeux de plateau,20,2,4,12,99
10545,10545,euuromind,€uromind,€uromind (1998),Le but du jeu est de faire le tour de l'Europe...,Jeux de plateau,60,2,6,8,99
10546,10546,gokiburipoka,ごきぶりポーカー,ごきぶりポーカー (2011),Un petit jeu de bluff où il faut éviter de réc...,"Animaux|Bluff|Collection / famille|Hasard (Dé,...",25,2,6,8,99
10547,10547,siehui-shephy,シェフィ - Shephy,シェフィ - Shephy (2013),Contenu : 72 cartes 1 mouton noir22 Actions7 p...,Animaux|Combinaison|Pioche|Programmation,15,1,1,<NA>,<NA>


In [39]:
jeux_clean.to_csv('../database_cleaned/jeux_clean.csv')

In [129]:
# separate number > 10 into first digit and the remainning ones
def digit_sep(n):
    nb_dig = int(np.log10(n))
    p = np.power(10, nb_dig)
    return n // p, n % p

condition = (jeux_clean["Min number of players"] >= 12) & (jeux_clean["Max number of players"].isna())

#for v in jeux_clean.loc[condition, "Min number of players"]:
    #print(v, digit_sep(v))
#jeux_clean_2 = jeux_clean.copy()
jeux_clean.loc[condition, ["Min number of players", "Max number of players"]] = jeux_clean.loc[condition, "Min number of players"].apply(lambda v : digit_sep(v)).to_list()
jeux_clean.to_csv("../database_cleaned/jeux_clean.csv")

In [130]:
jeux_clean

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
0,0,fever,$ Fever,$ Fever (1990),Le jeu comprend 6 dés dont les faces représent...,Jeux de dés,20,2,6,8,99
1,1,cetera,&cetera,&cetera (2003),&cetera est une extension de Roads & Boats qui...,Jeux de plateau,<NA>,1,6,14,99
2,2,43,'43,'43 (1981),'43 simule le débarquement en Sicile des force...,Jeux de guerre,240,2,4,12,99
3,3,und-tschuss,...Und Tschüss,...Und Tschüss (1997),But du jeu Remporter les cartes les plus forte...,Jeux de cartes,30,4,6,10,99
4,4,011,011,011 (2011),"L’histoire... Nous sommes en l’an 011, au XIX ...",Jeux de plateau,150,3,6,12,99
...,...,...,...,...,...,...,...,...,...,...,...
10544,10544,ilots,Îlots,Îlots (2010),Tout d'abord vous créez votre plateau de jeu g...,Jeux de plateau,20,2,4,12,99
10545,10545,euuromind,€uromind,€uromind (1998),Le but du jeu est de faire le tour de l'Europe...,Jeux de plateau,60,2,6,8,99
10546,10546,gokiburipoka,ごきぶりポーカー,ごきぶりポーカー (2011),Un petit jeu de bluff où il faut éviter de réc...,"Animaux|Bluff|Collection / famille|Hasard (Dé,...",25,2,6,8,99
10547,10547,siehui-shephy,シェフィ - Shephy,シェフィ - Shephy (2013),Contenu : 72 cartes 1 mouton noir22 Actions7 p...,Animaux|Combinaison|Pioche|Programmation,15,1,1,<NA>,<NA>


In [87]:
# jeux_clean_2.sort_values(by=["Min number of players"], ascending=False)
jeux_clean_2[jeux_clean["Min number of players"] > 12]
#print(jeux_clean.sort_values(by=["Min number of players"], ascending=False))
#jeux_clean[jeux_clean["Game id"] == 7595]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
681,681,au-bonheur-des-chevaux,Au bonheur des chevaux,Au bonheur des chevaux (2008),Pouvez-vous aider les enfants à retrouver leur...,Autres,15,2,4,5,99
1826,1826,cluedo-services-secrets,Cluedo - Services Secrets,Cluedo - Services Secrets (2009),Un jeu de société où vous recevez des indices ...,Jeux de plateau,<NA>,2,6,9,99
2562,2562,docteur-maboul-les-bons-voyages,Docteur Maboul - Les bons voyages,Docteur Maboul - Les bons voyages (2005),"Pour gagner, aidez votre patient à se débarras...",NaN,15,2,4,4,99
4326,4326,je-joue-je-lis,"Je joue, je lis...","Je joue, je lis... (1997)","Nathan a édité une série de jeux intitulés ""Je...",Autres,15,1,4,3,7
5634,5634,les-cyclistes,Les Cyclistes,Les Cyclistes,Contenu : . 5 coureurs . 5 billes . Une coupe ...,Autres,15,1,5,5,99
5672,5672,les-jeux-de-20-h,Les Jeux de 20 H,Les Jeux de 20 H,Une nouvelle version des célèbres jeux de 20 h...,Jeux de plateau,60,4,5,10,99
5686,5686,les-monsieur-madame-le-jeu,Les Monsieur Madame le jeu,Les Monsieur Madame le jeu (2009),Contenu : . 40 cartes défis (bleues) . 80 cart...,Jeux de plateau,20,2,6,3,8
5831,5831,ligretto-bob-l-eponge,Ligretto - Bob l'éponge,Ligretto - Bob l'éponge (2009),Aucune description,Jeux de cartes,15,2,5,8,99
7257,7257,pictureka-jeu-de-cartes,Pictureka! - jeu de cartes,Pictureka! - jeu de cartes (2009),"Pictureka version jeu de cartes, en format de ...",Jeux de cartes,15,2,5,6,99
7983,7983,rummikub-les-bons-voyages,Rummikub - Les bons voyages,Rummikub - Les bons voyages (2007),"Le Rummikub de Voyage, dans une boite métale. ...",NaN,30,2,4,8,99


In [93]:
jeux_clean.sort_values(by=["Min number of players"], ascending=False)[:30]
jeux_clean[(jeux_clean["Min number of players"] >= 10) & (~jeux_clean["Max number of players"].isna())]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
4027,4027,hollywood-lives,Hollywood Lives,Hollywood Lives (2005),Le jeu se déroule à l'âge d'or d'Hollywood (an...,Jeux de rôles,240,10,25,<NA>,<NA>


In [128]:
from collections import Counter
categories = [cat for cats in jeux_clean["Type"][~jeux_clean["Type"].isna()].str.split('|') for cat in cats]
count_cats = Counter(categories)
count_cats

Counter({'Jeux de plateau': 3163,
         'Jeux de cartes': 1742,
         'Jeux de guerre': 444,
         'Affrontement': 427,
         'Autres': 397,
         'Hasard (Dé, Cartes, ...)': 363,
         'Casse-tête': 297,
         'Placement': 285,
         'Combinaison': 263,
         'Déplacement': 257,
         'Ambiance': 250,
         'Jeux de pions': 244,
         'Médiéval-fantastique': 239,
         'Dés': 232,
         'Guerre': 226,
         'Gestion': 212,
         'Coopération': 210,
         'Gestion de main': 204,
         'Exploration': 201,
         'Animaux': 185,
         'Jeux de figurines': 185,
         'Jeu de carte à collectionner': 184,
         'Jeux de connaissances': 180,
         'Bluff': 179,
         'Fantastique': 172,
         'Majorité': 161,
         'Jeux de dés': 160,
         'Jeu de logique': 157,
         'Prise de risque': 153,
         'Science fiction': 153,
         'Commerce': 153,
         'Collection / famille': 150,
         'Médiéval': 1

In [124]:
id_games = jeux_clean[["Game id", "Game name UI"]].groupby(by="Game name UI").count()
id_games[id_games["Game id"] > 1]

jeux_clean[jeux_clean["Game name UI"] == "Waterloo"]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
10089,10089,waterloo-0,Waterloo,Waterloo (1981),Waterloo fait partie de la série International...,Jeux de guerre,240,2,3,12,99
10090,10090,waterloo-2,Waterloo,Waterloo (1996),Waterloo est la version française du jeu L'Arm...,Jeux de guerre,300,1,2,14,99
10091,10091,waterloo,Waterloo,Waterloo (2003),Idée du jeu : Waterloo : la dernière bataille ...,Jeux de guerre,60,2,3,12,99
10092,10092,waterloo-3,Waterloo,Waterloo (2009),Waterloo est le volume 4 de la Treefrog Line d...,Jeux de guerre,180,1,2,12,99


In [123]:
id_games[id_games["Game id"] > 1]


,Game id
Game name UI,
1830,2
1870,2
Age of Conan,2
Alcatraz,3
Alcazar,2
...,...
Warhammer : Rois des Tombes,2
Waterloo,4
Word-Whiz,2
